In [1]:
import db_conn
import pandas as pd
import re
pd.options.display.max_columns = None

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 부작용
추출된 전체 부작용 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_ads():
    cur = conn.cursor()
#     cur.execute("""
#     SELECT DISTINCT(TRIM(LOWER(d.tt_ad))) AS t_ad
#     FROM
#     (SELECT id, tt_ad
#     FROM 
#         (SELECT * 
#         FROM article_table_sentences_m
#         WHERE table_id IN 
#         (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
#         UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
#     WHERE (tt_ad='') is false) as d;
#     """)
    cur.execute("""
    SELECT id, TRIM(lower(d.tt_ad)) AS tt_ad
        FROM
        (SELECT id, tt_ad
        FROM 
            (SELECT * 
            FROM article_table_sentences_m
            WHERE table_id IN 
            (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
            UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
        WHERE (t_ad='') is false) as d;
        """)
    return cur.fetchall()

def get_ad_dictionary():
    cur = conn.cursor()
    cur.execute("""
    SELECT llt_code, pt_code, TRIM(LOWER(llt_name)) AS t_ad
    FROM meddra_llt_181022 M;
    """)
    return cur.fetchall()

In [4]:
ads = pd.DataFrame(get_total_ads())
ads.head()
# ads = pd.read_csv('./distinct_ads.tsv', sep='\t')

,0,1
0,2668800,adverse events
1,2668800,rash
2,2668803,adverse events
3,2668803,dry skin
4,2668802,adverse events


In [5]:
# ads.to_csv('./ads.csv', sep=',')

In [6]:
ads.columns=['id','name']

In [7]:
ads['refined'] = ''
ads['llt_code'] = None
ads['pt_code'] = None
ads.head()

,id,name,refined,llt_code,pt_code
0,2668800,adverse events,,None,None
1,2668800,rash,,None,None
2,2668803,adverse events,,None,None
3,2668803,dry skin,,None,None
4,2668802,adverse events,,None,None


#### 전처리 전 총 부작용의 수는 323398개 입니다

In [8]:
ads.shape

(323398, 5)

### 2. 전처리

#### A. dictionary를 참고하여, 정확히 일치하는 부작용 명은 전처리 없이 포함합니다.

In [9]:
dictionary = pd.DataFrame(get_ad_dictionary())
dictionary.columns = ['llt_code', 'pt_code', 't_ad']
# dictionary = pd.read_csv('./dictionary_ads.tsv', sep='\t')
# dictionary = list(map(lambda x: str(x[0]).lower().replace('"', ''), dictionary))

In [10]:
dictionary.shape

(60939, 3)

In [11]:
dictionary.head()

,llt_code,pt_code,t_ad
0,10000002,10000002,11-beta-hydroxylase deficiency
1,10000007,10000007,17 ketosteroids urine decreased
2,10000010,10000007,17 ketosteroids urine low
3,10000008,10000009,17 ketosteroids urine high
4,10000009,10000009,17 ketosteroids urine increased


### 부작용용어 dictionary와 정확이 매칭하면 추가 전처리 없이 확정

In [12]:
def apply_dictionary():
    for ind, ad in ads[ads['llt_code'].isnull()].iterrows():
    #     print(ad['name'].strip())
        name = ad['name'].strip()
        exact_mathing = None
        exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)==0:
            if name[0] in ['"', '.', ':', '-', '(', ')']:
                name = name[1:]
            if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
                name = name[:len(name)-1]
    #         print(name)

            exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)>0:
#             print(exact_mathing.values)
            ads.loc[ind, 'refined'] = name
            ads.loc[ind, 'llt_code'] = exact_mathing.values[0][0]
            ads.loc[ind, 'pt_code'] = exact_mathing.values[0][1]

In [13]:
apply_dictionary()

In [14]:
# ads[ads.refined=='apnea']

In [15]:
print(ads[ads['refined'] == ''].shape)
print(ads[ads['llt_code'].isnull()].shape)

(61980, 5)
(61980, 5)


#### A. 부작용을 포함하고 있는 구절이 추출 된 경우, 부작용 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_ads_hyj.xlsx파일을 통해 부작용명이 아니라고 지정해주신 목록을 추가하였습니다.

In [16]:
replace_with_space = ['beverages', '<70%', '%', 'values)', '(', ')', '-1987', '-1993', ':', '±', '↑', 'australia-new',
                     'scale-7', 'c-statistics', '-defined', '-derived', '-like', 'like', 'non-beverage',
                     'scale-0', '-days', 'fluorine-18-labelled', '-specific', 'co-variate', '-guided', '≥2°',
                     '≥1', 'composting', 'daids', 'decentralised', 'dmards', 'forage', 'full-scale', 'glues',
                     'granger', 'greyscale', 'grey-scale', 'harvesting', 'immersed', 'invariably',
                     '-guidance', 'lifelon', 'mwaura', 'naito', 'nesting', 'nomad', 'nonsystemi', '≥60', 'primers',
                     'prohibited', 'sanger', '-1993', '<50', '<02', '>heathly']

replace_ends_with = ['-related']

In [17]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in ads[ads['refined'] == ''].iterrows():
    refined = ''
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        ads.loc[ind, 'refined'] = refined.strip()
        print(row['name'])
        print(ads.loc[ind]['refined'])
    
    for end in replace_ends_with:
        if row['name'].endswith(end):
            ads.loc[ind, 'refined'] = row['name'].replace(end, '').strip()

ads.reset_index(drop=True)

aids-1987
aids
aids-1993
aids
aids-1987
aids
aids-1993
aids
aids-1987
aids
aids-1993
aids
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
influenza-like
influenza
influenza-like
influenza
acne-like
acne
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
influenza-like
influenza
influenza-like
influenza
influenza-like
influenza
aids-1987
aids
aids-1993
aids
asthma-like
asthma
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
acne-like
acne
acne-like
acne
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
infarction-defined
infarction
myoca

acne-like
acne
stenosis-specific
stenosis
hypertension-specific
hypertension
hypertension-specific
hypertension
influenza-like
influenza
influenza-like
influenza
aids-1993
aids
angina-guided
angina
influenza-like
influenza
influenza-like
influenza
hypertension-specific
hypertension
plasmacytoma-like
plasmacytoma
aids-1987
aids
aids-1993
aids
influenza-like
influenza
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
influenza-like
influenza
influenza-like
influenza
influenza-like
influenza
acne-like
acne
infarction-defined
infarction
myocardial infarction-defined
myocardial infarction
aids-1987
aids
aids-1993
aids
angina-guided
angina
variable:
variable
melanoma-specific
melanoma
influenza-like
influenza
influenza-like
influenza
angina-guided
angina
variable:
variable
influenza-like
influenza
serum-sickness-like
serum-sickness
aids-1987
aids
aids-1993
aids
headache-days
headache
infarction-defined
infarction
myocardial infarction-defined
myocardial infarc

,id,name,refined,llt_code,pt_code
0,2668800,adverse events,,None,None
1,2668800,rash,rash,10037844,10037844
2,2668803,adverse events,,None,None
3,2668803,dry skin,dry skin,10013786,10013786
4,2668802,adverse events,,None,None
5,2668802,yellow skin,yellow skin,10048245,10048245
6,2668816,abdominal wall abscess,abdominal wall abscess,10000099,10000099
7,2668816,adverse events,,None,None
8,2668826,adverse events,,None,None
9,2668826,dysphagia,dysphagia,10013950,10013950


#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [18]:
non_drug_ad_rel = ['-free', 'free-', ' free ', 'variable', 'wrong drug administered', 'exercise-induced',
                  'underdose', 'switch', 'massage', 'according', '-defining', 'arizona', 'controller', 'athletics',
                  'atopics', 'average', 'bachelor', 'award', '—gasoline', 'cards', 'comer', 'consisting', 'consumer',
                  'courage', 'covaria', 'coverage', 'dimers', 'distinguish', 'eavalues', 'emilia', 'multivaria',
                  'nsaids', 'preexisting', 'reaches', 'recording', 'restaura', 'spain', 'standard', 'tactics',
                  'tattoos', 'tasmania', 'telcagepant', 'testing', 'tragus', 'post-aids', 'scale–parent', 'body-mass-index',
                  'mass-cavity', 'non-melanoma skin', 'single-blind', 'non-comatose', 'non-wheezers', 'high-pitched',
                  'gadofluorine-m', 'anger-control', 'hiv-1-negative', 'pulmonary-systemic', '1apical', 'pre-infarction',
                  'non-trauma', 'stress-mass-heart', 'self-discharge', 'anti-japanese encephalitis', 'non-hypertension',
                   'non-infarct', 'post-stress', 'injury-donor', 'rate–pressure', 'non-neutropenic infections', 'non-hypertrophic',
                   'anti-tachycardia', 'scale-up', '5-fluorouracil', 'isar-smart', '18f-fluoride', 'post-acute myocardial infarction',
                   'hyper-responsiveness', 'pre-aids', 'mid-cavity', 'post-bleeding', 'scale–cognitive', 'double-blind',
                   'high-pressure', 'post-endocarditis', 'peri-procedural complications', 'modulation-blind', 'pre-shock',
                   'pre-angina', 'diphtheria–pertussis–tetanus', 'australia–new', 'q10values', 'pressure-time', 'non-congenital',
                   'anti-hypertensive', 'sepsis-induced', 'snaith-hamilton', 'mass–body', 'pain-to-balloon', 'pain-to-coronary',
                   'non-prostate cancer', 'cold-induced', 'non-melanoma', 'post-hypoxic', 'pre-injury', 'scale-16', 'non-atopic',
                   'non-hypoxic', 'cancer-norfolk', 'langerhans-cell', 'body-mass', 'double-blinding', 'fluorouracil-based',
                   'scale–child', 'fat-mass', 'nifurtimox-eflornithine', 'no adverse events', 'typical'] #'non'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [19]:
for ind, row in ads.iterrows():
    refined = ''
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        ads.drop(ind, inplace=True)
        print('===============================')

ads.reset_index(drop=True)

testing
testing
testing
telcagepant
telcagepant
telcagepant
telcagepant
telcagepant
telcagepant
telcagepant
telcagepant
telcagepant
telcagepant
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
covariate
nsaids
aids-defining
according
according
according
according
according
according
variable
multivariate
variable
variable
univariable
univariable
multivariable
univariable
multivariate
variable
variable
variable
variable
variable
multivariate
multivariate
multivariate
variable
variable
variable
variable
variable
variable
variable
variable
switchmrk
switchmrk
switchmrk
variable
variable
variable;
consisting
testing
covaria
body-mass
covariate
body-mass
covariate
covariate
according
standards
multivariate
standards
standards
standards
standards
standards
covariate

variable
variable
variable
variable
body-mass
multivariate
body-mass
multivariate
body-mass
multivariate
multivariate
multivariate
multivariate
multivariable
multivariate
body-mass
multivariate
variable
variable
multivariate
multivariate
variable
univariable
multivariate
multivariate
multivariate
variable
multivariate
variable
variable
multivariate
multivariable
multivariable
multivariate
multivariate
multivariate
multivariate
variable
univariable
multivariate
multivariate
multivariate
variable
multivariate
multivariable
variable
variable
multivariable
multivariable
multivariate
multivariate
variable
variable
multivariate-adjusted
multivariate
variable
variable
body-mass
multivariate
body-mass
multivariate
average
body-mass
multivariate
body-mass
multivariate
univariable
multivariable
variable
dependent-variable
multivariable
multivariate
multivariate
multivariate
multivariable
multivariable
variable
variable
univariable
univariable
multivariate
multivariate
variable
variable
variable


covariate
covariate
univariable
multivariate
multivariate
multivariate
covariate
multivariable
covariate
multivariable
multivariable
multivariate
multivariable
multivariate
variable
univariable
metastasis-free
multivariate
multivariable
multivariate
variable
covariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
variable
variable
multivariate
multivariate
variable
variable
multivariate
variable
variable
variable
variable
multivariate
multivariate
multivariate
multivariate
multivariate
univariable
multivariable
multivariable
variable
switch
switch
switch
switch
switch
switch
covariate
nsaids
multivariable
multivariable
multivariable
multivariable
multivariable
multivariable
multivariable
multivariable
multivariable
post-acute myocardial infarction
covariate
multivariate
multivariate
multivariate
multivariate
according
switch
double-blind
double-blind
double-blind
consisting
according
according
according

body-mass
multivariate
body-mass
mass–body
multivariable
body-mass
massage
body-mass
body-mass
body-mass
multivariate
body-mass
body-mass
body-mass
body-mass
multivariate
body-mass
variable
body-mass
multivariate
body-mass
multivariate
multivariable
multivariable
multivariate
body-mass
variable
covariate
body-mass
multivariate
body-mass
multivariate
body-mass
multivariate
courage
multivariable
body-mass
body-mass
body-mass
multivariable
variable
multivariate
multivariate
multivariable
multivariable
multivariable
multivariate
multivariate
multivariate
variable
variable
variable
univariable
multivariate
variable
variable
multivariate
univariable
post-bleeding
variable
univariable
univariable
multivariable
multivariate
variable
multivariable
univariable
univariable
multivariable
multivariable
variable
multivariable
variable
variable
multivariable
univariable
univariable
multivariable
univariable
univariable
multivariable
multivariable
multivariable
variable
variable
univariable
multivaria

multivariable
multivariable
multivariable
multivariate
multivariate
multivariate
multivariable
multivariable
univariable
multivariate
univariable
multivariable
multivariate
multivariable
multivariable
multivariable
multivariate
variable
variable
variable
covariate
covariate
covariate
covariate
multivariate
covariate
variable
variable
multivariable
variable
variable
variable
multivariate
multivariate
multivariate
multivariable
multivariate
multivariate
variable
variable
covariate
variable
multivariate
variable
univariable
multivariate
multivariate
multivariate
according
multivariable
according
multivariable
multivariate
multivariable
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
variable
variable
variable
variable
univariable
variable
univariable
univariable
multivariate
variable
variable
variable
multivariate
multivariate
variable
metastasis-free
multivariate
metastasis-free
multivariate
variable
multivariate
variable
univariable
covariate
u

...

variable
isar-smart
isar-smart
body-mass
multivariate
average
average
univariable
body-mass
non-neutropenic infections
double-blind
variable
multivariate
multivariate
pre-infarction
covaria
multivariate
multivariate
multivariate
multivariate
multivariable
multivariable
typical
multivariable
multivariate
multivariate
according
typically
multivariable
according
according
typically
multivariate
testing
testing
testing
testing
testing
testing
testing
testing
testing
testing
testing
testing
testing
according
multivariable
multivariable
multivariable
multivariable
variable
multivariable
average
multivariable
multivariable
variable
typical
typical
typical
typical
multivariable
typical
multivariable
multivariable
variable
multivariable
multivariable
multivariable
multivariable
multivariable
testing
variable
variable
switch
variable
variable
variable
multivariable
multivariable
multivariable
testing
variable
multivariate
multivariate
multivariate
multivariate
univariable
mass–body
multivariate


average
variable
multivariate
double-blind
multivariate
double-blind
body-mass
body-mass
body-mass
body-mass
multivariable
body-mass
body-mass
multivariate
average
average
variable
according
multivariable
fluorouracil-based
fluorouracil-based
fluorouracil-based
fluorouracil-based
underdosed
switched
multivariable
multivariable
multivariable
multivariable
multivariable
body-mass
multivariate
non-melanoma
non-melanoma skin
multivariate
multivariate
variable
variable
multivariable
multivariable
body-mass
multivariable
multivariable
multivariate
multivariable
multivariable
switched
switched
switched
switched
switched
switched
covariate
fluorouracil-based
non-melanoma
non-melanoma skin
non-melanoma
non-melanoma skin
univariable
univariable
multivariate
multivariable
multivariate
multivariable
multivariable
variable
univariable
variable
variable
multivariable
variable
multivariable
multivariable
multivariable
multivariable
multivariate
multivariate
multivariate
underdosed
underdosed
univaria

testing
variable
multivariate
univariable
multivariate
univariable
multivariate
multivariate
aids-defining
aids-defining
aids-defining
multivariable
body-mass
multivariate
multivariate
metastasis-free
multivariate
multivariate
multivariate
multivariate
multivariate
body-mass
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
multivariate
metastasis-free
multivariate
metastasis-free
body-mass
multivariate
multivariate
multivariate
variable
variable
multivariable
multivariable
multivariable
multivariable
variable
variable
multivariate
multivariate
multivariate
multivariate
multivariate
metastasis-free
according
body-mass
multivariate
body-mass
multivariate
average
average
variable
average
variable
variable
leukaemia-free
leukaemia-free
leukaemia-free
leukaemia-free
according
variable
double-blind
univariable
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass
body-mass

body-mass
according
according
according
according
according
multivariable
univariable
multivariable
variable
double-blind
average
according
according
switchmrk
testing
body-mass
metastasis-free
metastasis-free
multivariable
switchmrk
average
variable
variable
according
according
according
according
according
according
multivariate
testing
according
nifurtimox-eflornithine
nifurtimox-eflornithine
nifurtimox-eflornithine
according
nifurtimox-eflornithine
according
according
nifurtimox-eflornithine
body-mass
double-blind
according
according
testing
testing
nifurtimox-eflornithine
testing
testing
according
according
according
infarction-free
myocardial infarction-free
infarction-free
myocardial infarction-free
infarction-free
myocardial infarction-free
infarction-free
myocardial infarction-free
infarction-free
myocardial infarction-free
infarction-free
myocardial infarction-free
infarction-free
myocardial infarction-free
nifurtimox-eflornithine
double-blind
according
nifurtimox-eflornithin

,id,name,refined,llt_code,pt_code
0,2668800,adverse events,,None,None
1,2668800,rash,rash,10037844,10037844
2,2668803,adverse events,,None,None
3,2668803,dry skin,dry skin,10013786,10013786
4,2668802,adverse events,,None,None
5,2668802,yellow skin,yellow skin,10048245,10048245
6,2668816,abdominal wall abscess,abdominal wall abscess,10000099,10000099
7,2668816,adverse events,,None,None
8,2668826,adverse events,,None,None
9,2668826,dysphagia,dysphagia,10013950,10013950


In [20]:
ads.shape

(312819, 5)

### 파일 중에 부작용이 아니라고 표기해주신 것을 추가로 제거

In [21]:
replace_excel = pd.read_excel('./need_to_refined_ads_hyj.xlsx')
replace_excel.columns = ['key', 'replace_with', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.head()

,key,replace_with
0.0,anuloaortic ectasia,annuloaortic ectasia
14.0,epispadias-classical,epispadias
24.0,broncho-pneumonia,broncho pneumonia
40.0,cardio-myopathy,cardiomyopathy
50.0,hypertension-stroke,Hypertension + stroke


In [22]:
replace_excel.shape

(1218, 2)

In [23]:
for ind, cura in replace_excel[(replace_excel.replace_with.str.find('부작용 아님')>=0) | (replace_excel.replace_with.str.find('부작용아님')>=0)].iterrows():
#     print(cura['name'])
    need_to_drop = ads[ads.name.str.lower().str.find(cura['key'].lower())>=0]
    for i_n, d_n in need_to_drop.iterrows():
        ads.drop(i_n, inplace=True)
        
ads.reset_index(drop=True)

,id,name,refined,llt_code,pt_code
0,2668800,adverse events,,None,None
1,2668800,rash,rash,10037844,10037844
2,2668803,adverse events,,None,None
3,2668803,dry skin,dry skin,10013786,10013786
4,2668802,adverse events,,None,None
5,2668802,yellow skin,yellow skin,10048245,10048245
6,2668816,abdominal wall abscess,abdominal wall abscess,10000099,10000099
7,2668816,adverse events,,None,None
8,2668826,adverse events,,None,None
9,2668826,dysphagia,dysphagia,10013950,10013950


#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 274156 남습니다

In [24]:
ads.shape

(274156, 5)

### 복잡 부작용, 스펠링 미스 등을 선생님께서 주신 파일로 정제

In [25]:
def replace_with_manual_curation(ads):
    for ind, row in replace_excel[(replace_excel.replace_with.str.find('부작용 아님')<0) & (replace_excel.replace_with.str.find('부작용아님')<0)].iterrows():
        print(row)
        origin_str = ''
        for d_ind, d_row in ads[ads.llt_code.isnull()].loc[ads.name.str.strip()==row['key']].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'].lower()).strip()
            ads.loc[d_ind, 'refined'] = origin_str
        for d_ind, d_row in ads[ads.llt_code.isnull()].loc[ads.refined.str.strip()==row['key']].iterrows():
            origin_str = d_row['refined'].replace(row['key'], row['replace_with'].lower()).strip()
            ads.loc[d_ind, 'refined'] = origin_str
    return ads

ads = replace_with_manual_curation(ads)

key              anuloaortic ectasia
replace_with    annuloaortic ectasia
Name: 0.0, dtype: object
key             epispadias-classical
replace_with              epispadias
Name: 14.0, dtype: object
key             broncho-pneumonia
replace_with    broncho pneumonia
Name: 24.0, dtype: object
key             cardio-myopathy
replace_with     cardiomyopathy
Name: 40.0, dtype: object
key               hypertension-stroke
replace_with    Hypertension + stroke
Name: 50.0, dtype: object
key             mgh-adhd
replace_with        ADHD
Name: 64.0, dtype: object
key             heat-shock
replace_with         shock
Name: 77.0, dtype: object
key             neoplasm-associated
replace_with               neoplasm
Name: 86.0, dtype: object
key               neoangiogenesis-inflammation
replace_with    Neoangiogenesis + inflammation
Name: 114.0, dtype: object
key               dementia-motor neuron disease
replace_with    Dementia + motor neuron disease
Name: 132.0, dtype: object
key             v

key             coloboma-multiple
replace_with             coloboma
Name: 1830.0, dtype: object
key             hiv-12
replace_with       HIV
Name: 1841.0, dtype: object
key             non-acute coronary syndrome
replace_with           Troponin I increased
Name: 1861.0, dtype: object
key             generalised tonic-clonic seizures
replace_with     generalised tonic-clonic seizure
Name: 1879.0, dtype: object
key             factor-stemi
replace_with           STEMI
Name: 1883.0, dtype: object
key             dementia–symptom
replace_with            dementia
Name: 1897.0, dtype: object
key             stenosis≥50
replace_with       stenosis
Name: 1951.0, dtype: object
key             neuropathy-rasch-built
replace_with                neuropathy
Name: 1955.0, dtype: object
key             non-vertebral fractures
replace_with         multiple fractures
Name: 1958.0, dtype: object
key             fibro-lipomatosis
replace_with          lipomatosis
Name: 1981.0, dtype: object
key         

key             infarct-atypical
replace_with             infarct
Name: 3702.0, dtype: object
key             post-traumatic stress disorders
replace_with     Post-traumatic stress disorder
Name: 3713.0, dtype: object
key             infarction-4
replace_with         infarct
Name: 3731.0, dtype: object
key             riddle-nstemi
replace_with           nstemi
Name: 3749.0, dtype: object
key              stress-emotion
replace_with    stress symptoms
Name: 3828.0, dtype: object
key             adhd-dub1
replace_with         adhd
Name: 3856.0, dtype: object
key             dementia–comfort
replace_with            dementia
Name: 3858.0, dtype: object
key             irritable-bowel
replace_with    Irritable bowel
Name: 3869.0, dtype: object
key             ischaemia-driven
replace_with           ischaemia
Name: 3877.0, dtype: object
key             mismatch-hyperkinesia
replace_with             hyperkinesia
Name: 3883.0, dtype: object
key             all-causedeathorhospitalizationforhf

key             non-febrile convulsions
replace_with                 convulsion
Name: 5747.0, dtype: object
key             pre-procedural bleeding
replace_with                   bleeding
Name: 5794.0, dtype: object
key             injury-severity
replace_with             injury
Name: 5805.0, dtype: object
key             chronic kidney disease stage 3b
replace_with     Chronic kidney disease stage 3
Name: 5810.0, dtype: object
key             vi-tetanus
replace_with       tetanus
Name: 5832.0, dtype: object
key             pressure-sore
replace_with    Pressure sore
Name: 5881.0, dtype: object
key             gout2
replace_with     gout
Name: 5889.0, dtype: object
key             fibrillation-timi
replace_with         fibrillation
Name: 5903.0, dtype: object
key             meningitis-associated
replace_with               meningitis
Name: 5953.0, dtype: object
key             chronic-hepatitis
replace_with    chronic hepatitis
Name: 5997.0, dtype: object
key             cardiomyopathy

key             dementia-care
replace_with         dementia
Name: 7751.0, dtype: object
key                                post-trauma
replace_with    Post-traumatic stress disorder
Name: 7778.0, dtype: object
key             carcinoma-in-situ
replace_with    Carcinoma in situ
Name: 7818.0, dtype: object
key                          i.afib
replace_with    Atrial fibrillation
Name: 7820.0, dtype: object
key             ischemia-driven
replace_with           ischemia
Name: 7821.0, dtype: object
key             chest-pain
replace_with    chest pain
Name: 7836.0, dtype: object
key                 stress-left
replace_with    stress symptoms
Name: 7856.0, dtype: object
key             stent-thrombosis
replace_with          thrombosis
Name: 7876.0, dtype: object
key             thrombo-embolism
replace_with     Thromboembolism
Name: 7877.0, dtype: object
key             log-proteinuria
replace_with        proteinuria
Name: 7902.0, dtype: object
key             retro-orbital granuloma
replace_

key             nodal blocking
replace_with       nodal block
Name: 271.0, dtype: object
key             skin lesions
replace_with     skin lesion
Name: 279.0, dtype: object
key             sudden death?봭ot
replace_with        sudden death
Name: 285.0, dtype: object
key             uterine cancers
replace_with     uterine cancer
Name: 300.0, dtype: object
key             hepatitis adverse
replace_with            hepatitis
Name: 312.0, dtype: object
key             aches
replace_with     pain
Name: 337.0, dtype: object
key             febrile convulsions
replace_with     febrile convulsion
Name: 363.0, dtype: object
key             leg ulcers
replace_with     leg ulcer
Name: 371.0, dtype: object
key             delusional
replace_with      delusion
Name: 374.0, dtype: object
key             vertebral fractures
replace_with     vertebral fracture
Name: 378.0, dtype: object
key             mood changes
replace_with     mood change
Name: 390.0, dtype: object
key             microfilarial
r

key             minor bleeding
replace_with       Minor bleed
Name: 1453.0, dtype: object
key             glomerular filtration rate lower
replace_with      Glomerular filtration rate low
Name: 1455.0, dtype: object
key             poisonings
replace_with     poisoning
Name: 1501.0, dtype: object
key             exanthemas
replace_with     exanthema
Name: 1512.0, dtype: object
key             acute painful
replace_with       Acute pain
Name: 1513.0, dtype: object
key             depressionn
replace_with     depression
Name: 1536.0, dtype: object
key             biliary tract disorders
replace_with     biliary tract disorder
Name: 1543.0, dtype: object
key             intraprocedural hypotension
replace_with     Intraoperative hypotension
Name: 1549.0, dtype: object
key             menstrual disorders
replace_with     menstrual disorder
Name: 1559.0, dtype: object
key             pain in hands
replace_with     Pain in hand
Name: 1565.0, dtype: object
key             febrile seizures
rep

key             diseaseshock
replace_with           shock
Name: 2472.0, dtype: object
key             vascular disorders
replace_with     vascular disorder
Name: 2473.0, dtype: object
key             jaundiced
replace_with     Jaundice
Name: 2496.0, dtype: object
key             rage
replace_with    rage
Name: 2497.0, dtype: object
key             skin warming
replace_with       skin warm
Name: 2516.0, dtype: object
key             infarctions
replace_with     infarction
Name: 2541.0, dtype: object
key                       pneumocystis
replace_with    Pneumocystis pneumonia
Name: 2564.0, dtype: object
key             administration site reactions
replace_with     administration site reaction
Name: 2565.0, dtype: object
key             skin infections
replace_with     skin infection
Name: 2586.0, dtype: object
key             alcohol use disorders
replace_with     Alcohol use disorder
Name: 2588.0, dtype: object
key             internal herniation
replace_with             herniation
Na

key             peripheral vascular diseases
replace_with     peripheral vascular disease
Name: 3738.0, dtype: object
key                      leucoma
replace_with    Adherent leucoma
Name: 3745.0, dtype: object
key             with herniation
replace_with         herniation
Name: 3761.0, dtype: object
key             arthralgias
replace_with     arthralgia
Name: 3768.0, dtype: object
key             unstable angina pectoris
replace_with             unstable angina
Name: 3770.0, dtype: object
key             headaches
replace_with     headache
Name: 3778.0, dtype: object
key             atrial arrhythmias
replace_with     atrial arrhythmia
Name: 3783.0, dtype: object
key                      fractures
replace_with    multiple fractures
Name: 3785.0, dtype: object
key             palpitationsor
replace_with       palpitation
Name: 3814.0, dtype: object
key             herniation
replace_with    herniation
Name: 3823.0, dtype: object
key             chronic liver diseases
replace_with   

key             “central nervous system haemorrhages
replace_with      Central nervous system haemorrhage
Name: nan, dtype: object
key             “infusion site reactions”
replace_with       infusion site reaction
Name: nan, dtype: object
key              bradyarrhytmia
replace_with    bradyarrhythmia
Name: nan, dtype: object
key             pain gastrointestinal-related
replace_with            gastrointestinal pain
Name: nan, dtype: object
key             pain gastrointestinal
replace_with    gastrointestinal pain
Name: nan, dtype: object
key                     lues
replace_with    lues maligna
Name: nan, dtype: object
key             serum-sickness
replace_with    serum sickness
Name: nan, dtype: object
key             in-stent restenosis
replace_with       stent restenosis
Name: nan, dtype: object
key             non-epileptic seizures>healthy
replace_with             non-epileptic seizure
Name: nan, dtype: object
key              hernation
replace_with    herniation
Name: nan, dt

In [26]:
for ind, grp in ads[ads.llt_code.isnull()].groupby('refined'):
#     print(ind)
    found = dictionary.loc[dictionary.t_ad==ind.strip()]
    if len(found)>0:
#         print(found['cui_id'].values[0])
        ads.ix[grp.index.values, 'llt_code'] = found['llt_code'].values[0]
        ads.ix[grp.index.values, 'pt_code'] = found['pt_code'].values[0]

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


### 여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [27]:
def separate_unique_ads(ads):
    additional_ads = []

    for ind, row in ads[(ads.llt_code.isnull()) & (ads.refined.str.find('+')>=0)].iterrows():
        processed = None
        processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

        if len(processed)>0:
            ads.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'id':row['id'], 'refined':pro})

    for ind, row in ads[(ads.llt_code.isnull()) & (ads.refined.str.find(',')>=0)].iterrows():
        processed = None
        processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

        if len(processed)>0:
            ads.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'id':row['id'], 'refined':pro})


    ads = ads.append(pd.DataFrame(additional_ads))
#     ads.drop_duplicates(inplace=True)
    return ads

ads = separate_unique_ads(ads)

/home/grace/.local/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [28]:
ads.shape

(274216, 5)

In [29]:
# ads[ads.refined=='apnea']
# ads[ads.llt_code.isnull()]

### 추가 전처리가 필요한 약물 갯수

In [30]:
len(ads[(ads.refined=='')])

15049

In [31]:
def get_only_string(text):
    try:
        float(text)
        return False
    except ValueError:
        return True
    
def remove_comparison(name, spliter):
    arrs = list(filter(lambda x: get_only_string(x.strip().replace('%', '')) and x.strip(), name.split(spliter)))
    if len(arrs)>0:
        return arrs
    else:
        return None

ads = ads[~ads.index.duplicated(keep='first')]
for ind, row in ads[ads.refined==''].loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is None:
        ads.drop(ind, inplace=True)
ads.reset_index(drop=True)

for ind, row in ads[ads.refined==''].loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is not None:
        ads.loc[ind, 'refined'] = ''.join(processed)

In [32]:
ads.loc[ads.name.str.find('<')>=0]

,id,llt_code,name,pt_code,refined
1095,2138335,None,values<25,None,values
1104,2138342,None,values<15,None,values
1107,2138352,None,values<15,None,values
1109,2138391,None,values<25,None,values
1111,2138424,None,values<25,None,values
1185,2138499,None,values<25,None,values
1193,2138485,None,values<25,None,values
7861,286498,None,pneumonia<1,None,pneumonia
11623,2138387,None,values<25,None,values
11625,2138448,None,values<25,None,values


여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [33]:
additional_ads = []

for ind, row in ads[ads.refined==''].loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is None:
        ads.drop(ind, inplace=True)
ads.reset_index(drop=True)

for ind, row in ads[ads.refined==''].loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is not None:
        ads.loc[ind, 'refined'] = processed[0]
        for pro in processed[1:]:
            print(pro)
            additional_ads.append({'name':row['name'], 'id':row['id'], 'refined':pro})

ads = ads.append(pd.DataFrame(additional_ads))
# ads.drop_duplicates(inplace=True)

healthy


/home/grace/.local/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [34]:
ads.shape

(274161, 5)

In [35]:
# ads[ads.refined=='apnea']
ads[ads.llt_code.isnull()].shape

(16638, 5)

### 앞 뒤의 -, . 제거

In [36]:
remove_list = ['.', '-related', '-', ':', '+', '–', ';', '%', 'time', 'invariably', '||']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
ads.name = ads.name.map(lambda x: preprocess(x))

In [37]:
remove_exact = ["electrocardiogram changes", "discharged", "historyshock", "alternans", "nonmelanoma skin", "nonmelanoma", "values",
               "statistics", 'actuarial', 'dangerous', 'shockable', 'presence of q waves', 'australia', "topical", "systemic",
               "familial", "fluor", "variations", "towards", "slaughter", "prestroke", "variation", "adjusting", "storage", 
                "subscale", "approached", "reached", "discharges", "accidental", "morale", "australian", "teache", "accidents", 
                "overfitting", "persisting", "confusional", "typically", "existing", "scales", "yards", "typical", "australi", 
                "biomass", "onwards", "subscales", "univariatep", "unrestricted", "rewards", "variant", "mutivariate", "vegetarian", 
                "intolerance", "bivariate", "antibiotics", "adjusted", "cystatin", "hazards", "intense", "anginal", "elispot", 
                "blinded", "pregnant", "pressure", "antipsychotic", "deaths", "fluoropyrimidine", "tolerance", "variance", "cavity", 
                "fluorouracil", "transapical", "resting", "varia", "edwards", "fluorodeoxyglucose", "homocysteine", "teriflunomide", 
                "defibrillation", 'sudden death—not', 'burn—hot', 'beverages'
               ]
for i, row in ads.iterrows():
    if row['name'].strip().lower() in remove_exact:
        ads.drop(i, inplace=True)

ads.reset_index(drop=True)

,id,llt_code,name,pt_code,refined
0,2668800,10060933,adverse events,10060933,adverse event
1,2668800,10037844,rash,10037844,rash
2,2668803,10060933,adverse events,10060933,adverse event
3,2668803,10013786,dry skin,10013786,dry skin
4,2668802,10060933,adverse events,10060933,adverse event
5,2668802,10048245,yellow skin,10048245,yellow skin
6,2668816,10000099,abdominal wall abscess,10000099,abdominal wall abscess
7,2668816,10060933,adverse events,10060933,adverse event
8,2668826,10060933,adverse events,10060933,adverse event
9,2668826,10013950,dysphagia,10013950,dysphagia


In [38]:
apply_dictionary()

In [39]:
# ads.drop_duplicates(subset=['id', 'llt_code', 'name', 'pt_code', 'refined'], inplace=True)
# ads.reset_index(drop=True)
# ads.head()
ads[ads.refined=='apnea']

,id,llt_code,name,pt_code,refined
20368,410874,None,sleep apnoea-hypopnoea,None,apnea
46536,247295,10002972,apnea,10002974,apnea
61660,410906,None,sleep apnoea-hypopnoea,None,apnea
69121,410868,None,sleep apnoea-hypopnoea,None,apnea
69890,410899,None,sleep apnoea-hypopnoea,None,apnea
92680,410872,None,sleep apnoea-hypopnoea,None,apnea
93959,410876,None,sleep apnoea-hypopnoea,None,apnea
102669,247294,10002972,apnea,10002974,apnea
105756,1419006,10002972,apnea,10002974,apnea
105757,1419006,None,apnea-hypopnea,None,apnea


In [40]:
ads = ads[~ads.index.duplicated(keep='first')]
ads = replace_with_manual_curation(ads)

key              anuloaortic ectasia
replace_with    annuloaortic ectasia
Name: 0.0, dtype: object
key             epispadias-classical
replace_with              epispadias
Name: 14.0, dtype: object
key             broncho-pneumonia
replace_with    broncho pneumonia
Name: 24.0, dtype: object
key             cardio-myopathy
replace_with     cardiomyopathy
Name: 40.0, dtype: object
key               hypertension-stroke
replace_with    Hypertension + stroke
Name: 50.0, dtype: object
key             mgh-adhd
replace_with        ADHD
Name: 64.0, dtype: object
key             heat-shock
replace_with         shock
Name: 77.0, dtype: object
key             neoplasm-associated
replace_with               neoplasm
Name: 86.0, dtype: object
key               neoangiogenesis-inflammation
replace_with    Neoangiogenesis + inflammation
Name: 114.0, dtype: object
key               dementia-motor neuron disease
replace_with    Dementia + motor neuron disease
Name: 132.0, dtype: object
key             v

key             coloboma-multiple
replace_with             coloboma
Name: 1830.0, dtype: object
key             hiv-12
replace_with       HIV
Name: 1841.0, dtype: object
key             non-acute coronary syndrome
replace_with           Troponin I increased
Name: 1861.0, dtype: object
key             generalised tonic-clonic seizures
replace_with     generalised tonic-clonic seizure
Name: 1879.0, dtype: object
key             factor-stemi
replace_with           STEMI
Name: 1883.0, dtype: object
key             dementia–symptom
replace_with            dementia
Name: 1897.0, dtype: object
key             stenosis≥50
replace_with       stenosis
Name: 1951.0, dtype: object
key             neuropathy-rasch-built
replace_with                neuropathy
Name: 1955.0, dtype: object
key             non-vertebral fractures
replace_with         multiple fractures
Name: 1958.0, dtype: object
key             fibro-lipomatosis
replace_with          lipomatosis
Name: 1981.0, dtype: object
key         

key             infarct-atypical
replace_with             infarct
Name: 3702.0, dtype: object
key             post-traumatic stress disorders
replace_with     Post-traumatic stress disorder
Name: 3713.0, dtype: object
key             infarction-4
replace_with         infarct
Name: 3731.0, dtype: object
key             riddle-nstemi
replace_with           nstemi
Name: 3749.0, dtype: object
key              stress-emotion
replace_with    stress symptoms
Name: 3828.0, dtype: object
key             adhd-dub1
replace_with         adhd
Name: 3856.0, dtype: object
key             dementia–comfort
replace_with            dementia
Name: 3858.0, dtype: object
key             irritable-bowel
replace_with    Irritable bowel
Name: 3869.0, dtype: object
key             ischaemia-driven
replace_with           ischaemia
Name: 3877.0, dtype: object
key             mismatch-hyperkinesia
replace_with             hyperkinesia
Name: 3883.0, dtype: object
key             all-causedeathorhospitalizationforhf

key             non-febrile convulsions
replace_with                 convulsion
Name: 5747.0, dtype: object
key             pre-procedural bleeding
replace_with                   bleeding
Name: 5794.0, dtype: object
key             injury-severity
replace_with             injury
Name: 5805.0, dtype: object
key             chronic kidney disease stage 3b
replace_with     Chronic kidney disease stage 3
Name: 5810.0, dtype: object
key             vi-tetanus
replace_with       tetanus
Name: 5832.0, dtype: object
key             pressure-sore
replace_with    Pressure sore
Name: 5881.0, dtype: object
key             gout2
replace_with     gout
Name: 5889.0, dtype: object
key             fibrillation-timi
replace_with         fibrillation
Name: 5903.0, dtype: object
key             meningitis-associated
replace_with               meningitis
Name: 5953.0, dtype: object
key             chronic-hepatitis
replace_with    chronic hepatitis
Name: 5997.0, dtype: object
key             cardiomyopathy

key             dementia-care
replace_with         dementia
Name: 7751.0, dtype: object
key                                post-trauma
replace_with    Post-traumatic stress disorder
Name: 7778.0, dtype: object
key             carcinoma-in-situ
replace_with    Carcinoma in situ
Name: 7818.0, dtype: object
key                          i.afib
replace_with    Atrial fibrillation
Name: 7820.0, dtype: object
key             ischemia-driven
replace_with           ischemia
Name: 7821.0, dtype: object
key             chest-pain
replace_with    chest pain
Name: 7836.0, dtype: object
key                 stress-left
replace_with    stress symptoms
Name: 7856.0, dtype: object
key             stent-thrombosis
replace_with          thrombosis
Name: 7876.0, dtype: object
key             thrombo-embolism
replace_with     Thromboembolism
Name: 7877.0, dtype: object
key             log-proteinuria
replace_with        proteinuria
Name: 7902.0, dtype: object
key             retro-orbital granuloma
replace_

key             skin lesions
replace_with     skin lesion
Name: 279.0, dtype: object
key             sudden death?봭ot
replace_with        sudden death
Name: 285.0, dtype: object
key             uterine cancers
replace_with     uterine cancer
Name: 300.0, dtype: object
key             hepatitis adverse
replace_with            hepatitis
Name: 312.0, dtype: object
key             aches
replace_with     pain
Name: 337.0, dtype: object
key             febrile convulsions
replace_with     febrile convulsion
Name: 363.0, dtype: object
key             leg ulcers
replace_with     leg ulcer
Name: 371.0, dtype: object
key             delusional
replace_with      delusion
Name: 374.0, dtype: object
key             vertebral fractures
replace_with     vertebral fracture
Name: 378.0, dtype: object
key             mood changes
replace_with     mood change
Name: 390.0, dtype: object
key             microfilarial
replace_with       filariasis
Name: 410.0, dtype: object
key             petechial
replace

key             poisonings
replace_with     poisoning
Name: 1501.0, dtype: object
key             exanthemas
replace_with     exanthema
Name: 1512.0, dtype: object
key             acute painful
replace_with       Acute pain
Name: 1513.0, dtype: object
key             depressionn
replace_with     depression
Name: 1536.0, dtype: object
key             biliary tract disorders
replace_with     biliary tract disorder
Name: 1543.0, dtype: object
key             intraprocedural hypotension
replace_with     Intraoperative hypotension
Name: 1549.0, dtype: object
key             menstrual disorders
replace_with     menstrual disorder
Name: 1559.0, dtype: object
key             pain in hands
replace_with     Pain in hand
Name: 1565.0, dtype: object
key             febrile seizures
replace_with     febrile seizure
Name: 1574.0, dtype: object
key             aortic aneurysms
replace_with    aortic aneurysm 
Name: 1575.0, dtype: object
key             colon cancers
replace_with     colon cancer
Name

key             rage
replace_with    rage
Name: 2497.0, dtype: object
key             skin warming
replace_with       skin warm
Name: 2516.0, dtype: object
key             infarctions
replace_with     infarction
Name: 2541.0, dtype: object
key                       pneumocystis
replace_with    Pneumocystis pneumonia
Name: 2564.0, dtype: object
key             administration site reactions
replace_with     administration site reaction
Name: 2565.0, dtype: object
key             skin infections
replace_with     skin infection
Name: 2586.0, dtype: object
key             alcohol use disorders
replace_with     Alcohol use disorder
Name: 2588.0, dtype: object
key             internal herniation
replace_with             herniation
Name: 2616.0, dtype: object
key                 supraventricular arrhythmia
replace_with    Supraventricular arrhythmia NOS
Name: 2629.0, dtype: object
key             cardiac deaths
replace_with     cardiac death
Name: 2630.0, dtype: object
key             gastroin

key             arthralgias
replace_with     arthralgia
Name: 3768.0, dtype: object
key             unstable angina pectoris
replace_with             unstable angina
Name: 3770.0, dtype: object
key             headaches
replace_with     headache
Name: 3778.0, dtype: object
key             atrial arrhythmias
replace_with     atrial arrhythmia
Name: 3783.0, dtype: object
key                      fractures
replace_with    multiple fractures
Name: 3785.0, dtype: object
key             palpitationsor
replace_with       palpitation
Name: 3814.0, dtype: object
key             herniation
replace_with    herniation
Name: 3823.0, dtype: object
key             chronic liver diseases
replace_with     chronic liver disease
Name: 3828.0, dtype: object
key             headcache
replace_with     headache
Name: 3833.0, dtype: object
key             ?쐇nfusion site reactions??
replace_with        Infusion site reaction
Name: 3838.0, dtype: object
key              cardiorespiratory arrest
replace_with    

key              bradyarrhytmia
replace_with    bradyarrhythmia
Name: nan, dtype: object
key             pain gastrointestinal-related
replace_with            gastrointestinal pain
Name: nan, dtype: object
key             pain gastrointestinal
replace_with    gastrointestinal pain
Name: nan, dtype: object
key                     lues
replace_with    lues maligna
Name: nan, dtype: object
key             serum-sickness
replace_with    serum sickness
Name: nan, dtype: object
key             in-stent restenosis
replace_with       stent restenosis
Name: nan, dtype: object
key             non-epileptic seizures>healthy
replace_with             non-epileptic seizure
Name: nan, dtype: object
key              hernation
replace_with    herniation
Name: nan, dtype: object
key             lactic-acidosis-related
replace_with            lactic acidosis
Name: nan, dtype: object
key             lactic-acidosis
replace_with    lactic acidosis
Name: nan, dtype: object
key                          cardi

In [41]:
ads = separate_unique_ads(ads)
ads.reset_index(drop=True)

/home/grace/.local/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,id,llt_code,name,pt_code,refined
0,2668800,10060933,adverse events,10060933,adverse event
1,2668800,10037844,rash,10037844,rash
2,2668803,10060933,adverse events,10060933,adverse event
3,2668803,10013786,dry skin,10013786,dry skin
4,2668802,10060933,adverse events,10060933,adverse event
5,2668802,10048245,yellow skin,10048245,yellow skin
6,2668816,10000099,abdominal wall abscess,10000099,abdominal wall abscess
7,2668816,10060933,adverse events,10060933,adverse event
8,2668826,10060933,adverse events,10060933,adverse event
9,2668826,10013950,dysphagia,10013950,dysphagia


In [42]:
for ind, grp in ads[ads.llt_code.isnull()].groupby('refined'):
#     print(ind)
    found = dictionary.loc[dictionary.t_ad==ind.strip()]
    if len(found)>0:
#         print(found['cui_id'].values[0])
        ads.ix[grp.index.values, 'llt_code'] = found['llt_code'].values[0]
        ads.ix[grp.index.values, 'pt_code'] = found['pt_code'].values[0]

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [43]:
for ind, grp in ads[ads.llt_code.isnull()].groupby('name'):
#     print(ind)
    found = dictionary.loc[dictionary.t_ad==ind.strip()]
    if len(found)>0:
#         print(found['cui_id'].values[0])
        ads.ix[grp.index.values, 'refined'] = ind.strip()
        ads.ix[grp.index.values, 'llt_code'] = found['llt_code'].values[0]
        ads.ix[grp.index.values, 'pt_code'] = found['pt_code'].values[0]

In [44]:
# ads[(ads.llt_code=='') | (ads.pt_code=='')]['refined'].tolist()
# 'illness', 'wind', 'sting','tics','nstemi','herniation','suffering', 'fasting','carcinomtous rash','apoptosis',
irrelevant_words = ['withdrawn', 'non-ischaemia', 'illness', 'wind', 'values', 'sting', 'pressure', 'nstemi', 'rage',\
 'hanging', 'windy', 'tics', 'herniation','suffering', 'fasting','carcinomtous rash','apoptosis', 'term birth']
for ind, row in ads[(ads.llt_code.isnull()) | (ads.pt_code.isnull())].iterrows():
    refined = row['refined']
    if refined.strip() in irrelevant_words:
        ads.drop(ind, inplace=True)

In [45]:
ads.reset_index(drop=True)

,id,llt_code,name,pt_code,refined
0,2668800,10021079,adverse events,10021079,adverse event
1,2668800,10021079,rash,10021079,rash
2,2668803,10021079,adverse events,10021079,adverse event
3,2668803,10021079,dry skin,10021079,dry skin
4,2668802,10021079,adverse events,10021079,adverse event
5,2668802,10021079,yellow skin,10021079,yellow skin
6,2668816,10021079,abdominal wall abscess,10021079,abdominal wall abscess
7,2668816,10021079,adverse events,10021079,adverse event
8,2668826,10021079,adverse events,10021079,adverse event
9,2668826,10021079,dysphagia,10021079,dysphagia


In [46]:
# ads[ads.llt_code.isnull()].refined = ads[ads.llt_code.isnull()].refined.apply(lambda x: x.strip())

In [47]:
ads[ads.llt_code.isnull()]

,id,llt_code,name,pt_code,refined


In [49]:
ads[ads.refined=='apnea']

,id,llt_code,name,pt_code,refined
20368,410874,10002972,sleep apnoea-hypopnoea,10002974,apnea
46536,247295,10002972,apnea,10002974,apnea
61660,410906,10002972,sleep apnoea-hypopnoea,10002974,apnea
69121,410868,10002972,sleep apnoea-hypopnoea,10002974,apnea
69890,410899,10002972,sleep apnoea-hypopnoea,10002974,apnea
92680,410872,10002972,sleep apnoea-hypopnoea,10002974,apnea
93959,410876,10002972,sleep apnoea-hypopnoea,10002974,apnea
102669,247294,10002972,apnea,10002974,apnea
105756,1419006,10002972,apnea,10002974,apnea
105757,1419006,10002972,apnea-hypopnea,10002974,apnea


In [50]:
ads[ads.llt_code.isnull()].refined.unique()#.shape

array([], dtype=object)

In [51]:
# ads[(ads.llt_code=='') | (ads.pt_code.str=='')]
ads.isnull().sum()

id          0
llt_code    0
name        0
pt_code     0
refined     0
dtype: int64

In [52]:
ads.shape

(257713, 5)

In [53]:
ads[~ads.llt_code.isnull()].shape

(257713, 5)

## Bulk insert

In [54]:
import os
from io import StringIO

In [55]:
ads.head()

,id,llt_code,name,pt_code,refined
0,2668800,10021079,adverse events,10021079,adverse event
1,2668800,10021079,rash,10021079,rash
2,2668803,10021079,adverse events,10021079,adverse event
3,2668803,10021079,dry skin,10021079,dry skin
4,2668802,10021079,adverse events,10021079,adverse event


In [56]:
ads.columns

Index(['id', 'llt_code', 'name', 'pt_code', 'refined'], dtype='object')

In [57]:
ads.columns = ['sent_id', 'llt_code', 'name', 'pt_code', 'refined']
ads.head()

,sent_id,llt_code,name,pt_code,refined
0,2668800,10021079,adverse events,10021079,adverse event
1,2668800,10021079,rash,10021079,rash
2,2668803,10021079,adverse events,10021079,adverse event
3,2668803,10021079,dry skin,10021079,dry skin
4,2668802,10021079,adverse events,10021079,adverse event


In [58]:
sio = StringIO()
sio.write(ads.to_csv(index=None, header=None))
sio.seek(0)

with conn.cursor() as c:
    c.copy_from(sio, "table_extracted_ad", columns = ads.columns, sep=',')
    conn.commit()

In [59]:
# ads[(ads.llt_code=='') | (ads.pt_code.str=='')]
#ads[(ads.id.isnull())]
ads.shape

(257713, 5)

In [60]:
ads[ads.refined=='apnea']

,sent_id,llt_code,name,pt_code,refined
20368,410874,10002972,sleep apnoea-hypopnoea,10002974,apnea
46536,247295,10002972,apnea,10002974,apnea
61660,410906,10002972,sleep apnoea-hypopnoea,10002974,apnea
69121,410868,10002972,sleep apnoea-hypopnoea,10002974,apnea
69890,410899,10002972,sleep apnoea-hypopnoea,10002974,apnea
92680,410872,10002972,sleep apnoea-hypopnoea,10002974,apnea
93959,410876,10002972,sleep apnoea-hypopnoea,10002974,apnea
102669,247294,10002972,apnea,10002974,apnea
105756,1419006,10002972,apnea,10002974,apnea
105757,1419006,10002972,apnea-hypopnea,10002974,apnea


In [61]:
ads[ads.name=='seizures>healthy']
# ads.loc[0, 'id']= 2274584

,sent_id,llt_code,name,pt_code,refined
18402,2274584,10039910,seizures>healthy,10039906,seizures


### 추가 전처리 필요없는 부작용들은  refined로 입력

In [ ]:
# remove_list.extend([')', '('])
# spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
# sep_pattern = re.compile("|".join(spe_chars.keys()))
# int_pattern = re.compile('[0-9]')

# for ind, row in ads.iterrows():
#     refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
#     int_refined = int_pattern.search(row['name'])
    
#     if refined.strip() == '':
#         ads.drop(ind, inplace=True)
#     elif row['name'].strip() == refined.strip() and int_refined is None:
#         ads.loc[ind]['refined'] = refined
#         print(row['name'])
#         print(ads.loc[ind]['refined'])
#     print('===============================')

In [ ]:
# ads.loc[ads.refined.str.len()>0].shape

In [ ]:
# ads.reset_index(drop=True)

In [ ]:
ads[ads.name.str.find('sudden death')>=0]

## 정체된 목록입니다.

In [ ]:
# ads.loc[ads.refined.str.strip()!=''].to_csv('./refined_ads_0226.csv', sep=',')

## 선생님께 확인 요청을 드리고 싶은 부분

In [ ]:
# ads[ads['refined'] == ''].to_csv('./need_to_refined_ads_0226.csv', sep=',')